# EDA variables nuevas diabetes


##### Jesus Camargo

In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Aparencia general graficos

In [ ]:
%matplotlib inline
sns.set_style(style="whitegrid")
sns.set_context(context='notebook')
plt.rcParams['figure.figsize']=(9,8)



In [ ]:
# Datos de las tablas a traer
proyecto = 'co-keralty-costomedico'
dataset = 'Diabetes_avicena_new'

# Nombres de tablas disponibles en el dataset
tables = ['diabetes', 'examenes_por_usuarios','antecedentes_x_usuario','perimetros_x_usuarios','actividadFisica_x_usuarios','diabetes_copy']

client_bq = bigquery.Client(project= proyecto)

In [ ]:
def bring_data_from_bq(table, client_bq, query = '', save = False, read_local = False):

    print(f'Leyendo datos de la tabla: {table}')

    if query == '':
        query = f"""SELECT * FROM {proyecto}.{dataset}.{table} WHERE edad >= 18"""

    if read_local:
        data = pd.read_parquet(f'{table}.parquet')
    else:
        data = client_bq.query(query).result().to_dataframe()

        if save:
            data.to_parquet(f'{table}.parquet')
            print(f'Informacion guardada en el archivo: {table}.parquet')

    return data


In [ ]:
data_original = bring_data_from_bq(table = tables[0], client_bq = client_bq, save = True, read_local = True)
data_original

### Tipo de datos

In [ ]:
data_original.dtypes

In [ ]:
# conversion de datos
data_original["peso"]=data_original["peso"].astype(float)
data_original["talla"]=data_original["talla"].astype(float)
data_original["imc"]=data_original["imc"].astype(float)
data_original["HDL"]=data_original["HDL"].astype(float)
data_original["LDL"]=data_original["LDL"].astype(float)
data_original["PERIMETRO_ABDOMINAL"]=data_original["PERIMETRO_ABDOMINAL"].astype(float)
data_original["trigliceridos"]=data_original["trigliceridos"].astype(float)
data_original["hace_ejercicio_min"]=data_original.hace_ejercicio.str.extract(r'(\d+)').fillna(0).astype(float)

### Cantidad de variables por tipo de dato


In [ ]:
data_original.dtypes.value_counts()

### Cantidad registro y observaciones

In [ ]:
print(f"Cantidad de registros en el data set: {data_original.shape[0]:,}")
print(f"Cantidad de columnas en el data set: {data_original.shape[1]}")

### Valores duplicados.

In [ ]:
data_original.diabetes.value_counts()

In [ ]:
data_original[data_original.drop(["hace_ejercicio","hace_ejercicio_min"],axis=1).duplicated()]#.diabetes.value_counts()

In [ ]:
data_original[data_original['diabetes']==0]['numero_identificacion_paciente'].value_counts()

In [ ]:
data_original.drop(["hace_ejercicio","hace_ejercicio_min"],axis=1)[data_original['numero_identificacion_paciente']=="1519880"]#.drop_duplicates()

### Valores nulos

definición de albuminuria 

- La albuminuria es un signo de enfermedad renal y significa que el paciente tiene exceso de albúmina en la orina. 
	La albúmina es una proteína que se encuentra en la sangre. Un riñón sano no permite que la albúmina pase de la sangre a la orina, 
	mientras que un riñón dañado sí deja pasar algo de albúmina a la orina.

In [ ]:
nulos = data_original.isnull()
nulos.any()

In [ ]:
nulos.sum()

In [ ]:
print(f"Cantidad de valores nulos en todo el dataset: {nulos.sum().sum():,}")

In [ ]:
nulos.melt().pipe(
	lambda df: (
		sns.displot(
			data=df,
			y="variable",
			hue="value",
			multiple='fill',
			aspect=2
		)
	)
)

In [ ]:
nulos.transpose().pipe( lambda df: sns.heatmap(data=df))

# informacion que perdemos si eliminamos los datos faltantes

In [ ]:
count_elemento_org = data_original.shape[0]
count_elemento_no_nan = data_original.dropna().shape[0]

print(" cantidad de datos elimindados", count_elemento_org-count_elemento_no_nan)



## descripcion de la base

1. HDL (High-Density Lipoprotein) - "Colesterol Bueno":  
¿Qué es?: El HDL es conocido como el "colesterol bueno" porque ayuda a eliminar el colesterol malo (LDL) de las arterias, llevándolo al hígado para ser eliminado del cuerpo.  
Rangos:  
Normal: Hombres: ≥ 40 mg/dL | Mujeres: ≥ 50 mg/dL.  
Alerta: Si es bajo (Hombres < 40 mg/dL, Mujeres < 50 mg/dL), el riesgo de enfermedad cardíaca aumenta.  
Muy bueno: ≥ 60 mg/dL se considera protector contra enfermedades cardíacas.  

2. LDL (Low-Density Lipoprotein) - "Colesterol Malo":  
¿Qué es?: El LDL es el "colesterol malo" porque puede acumularse en las arterias, formando placas que pueden bloquear el flujo sanguíneo, aumentando el riesgo de ataques cardíacos y accidentes cerebrovasculares.  
Rangos:  
Normal: < 100 mg/dL es ideal.  
Límite Alto: 130-159 mg/dL.  
Alto: 160-189 mg/dL.  
Alerta: ≥ 190 mg/dL es muy alto y peligroso para la salud cardiovascular.  
3. Triglicéridos:  
¿Qué son?: Los triglicéridos son un tipo de grasa (lípido) en la sangre. Cuando comes, tu cuerpo convierte las calorías que no necesitas en triglicéridos, que luego se almacenan en las células de grasa.  
Rangos:  
Normal: < 150 mg/dL.  
Límite Alto: 150-199 mg/dL.  
Alto: 200-499 mg/dL.  
Alerta: ≥ 500 mg/dL, aumenta el riesgo de pancreatitis y problemas cardíacos.  
4. Índice de Masa Corporal (IMC):  
¿Qué es?: El IMC es una medida que se usa para evaluar si una persona tiene un peso saludable en relación con su altura. Se calcula dividiendo el peso (en kilogramos) por el cuadrado de la altura (en metros).  
Rangos:  
Bajo peso: IMC < 18.5.  
Normal: IMC entre 18.5 y 24.9.  
Sobrepeso: IMC entre 25 y 29.9.  
Obesidad: IMC ≥ 30.  
Alerta: Un IMC elevado (sobrepeso u obesidad) aumenta el riesgo de enfermedades como la diabetes tipo 2, enfermedades cardíacas y ciertos tipos de cáncer.  

### funcion etiquetas

In [ ]:
# funcion auxiliar
def asignar_etiquetas(df):
    # Etiquetas para HDL
    def etiqueta_hdl(value, gender):
        
        if gender == 0:
            if value == 0:
                return 'No tiene'
            if value >= 60:
                return 'Muy bueno'
            elif value >= 40:
                return 'Normal'
            else:
                return 'Alerta'
        elif gender == 1:
            if value == 0:
                return 'No tiene'
            if value >= 60:
                return 'Muy bueno'
            elif value >= 50:
                return 'Normal'
            else:
                return 'Alerta'

    # Etiquetas para LDL
    def etiqueta_ldl(value):
        if value < 1:
            return 'No tiene'
        elif 1 <= value < 100:
            return 'Normal'
        elif 100 <= value < 130:
            return 'Casi normal'
        elif 130 <= value < 160:
            return 'Límite alto'
        elif 160 <= value < 190:
            return 'Alto'
        else:
            return 'Alerta'

    # Etiquetas para Triglicéridos
    def etiqueta_trigliceridos(value):
        if value < 1:
            return 'No tiene'
        elif 1 <= value < 150:
            return 'Normal'
        elif 150 <= value < 200:
            return 'Límite alto'
        elif 200 <= value < 500:
            return 'Alto'
        else:
            return 'Alerta'

    # Etiquetas para IMC
    def etiqueta_imc(value):
        if value < 18.5:
            return 'Bajo peso'
        elif 18.5 <= value < 25:
            return 'Normal'
        elif 25 <= value < 30:
            return 'Sobrepeso'
        else:
            return 'Obesidad'

    # Crear nuevas columnas en el DataFrame con las etiquetas asignadas
    df['Etiqueta_HDL'] = df.apply(lambda row: etiqueta_hdl(row['HDL'], row['genero']), axis=1)
    df['Etiqueta_LDL'] = df['LDL'].apply(etiqueta_ldl)
    df['Etiqueta_Triglicéridos'] = df['trigliceridos'].apply(etiqueta_trigliceridos)
    df['Etiqueta_IMC'] = df['imc'].apply(etiqueta_imc)

    return df

In [ ]:
100/(1.68**2)

In [ ]:
data_original.drop(["hace_ejercicio","hace_ejercicio_min"],axis=1)[data_original.drop(["hace_ejercicio","hace_ejercicio_min"],axis=1).duplicated()]

In [ ]:
data_original.numero_identificacion_paciente.value_counts()

In [ ]:
data_original[data_original.numero_identificacion_paciente=="63447134"]

In [ ]:
df=asignar_etiquetas(data_original.drop(["hace_ejercicio","hace_ejercicio_min"],axis=1)[~data_original.drop(["hace_ejercicio","hace_ejercicio_min"],axis=1).duplicated()])

In [ ]:
df.describe(include="all").iloc[:,:10]

In [ ]:
df.describe(include="all").iloc[:,10:]

In [ ]:
df.describe()

In [ ]:
df.describe(include=object)

### Graficas

In [ ]:
sns.catplot(
	data=df,
 x="Etiqueta_HDL",
 kind="count",
 hue="diabetes"
)

In [ ]:
sns.catplot(
	data=df,
 x="Etiqueta_LDL",
 kind="count",
 hue="diabetes"
)

In [ ]:
sns.catplot(
	data=df,
 x="Etiqueta_Triglicéridos",
 kind="count",
 hue="diabetes"
)

In [ ]:
sns.catplot(
	data=df,
 x="Etiqueta_IMC",
 kind="count",
 hue="diabetes"
)

In [ ]:
sns.catplot(
	data=df[df.dm_gestacional==1],
 x="dm_gestacional",
 kind="count",
 hue="diabetes"
)

In [ ]:
# Definir los intervalos (bins) y las etiquetas
bins = [ 18, 35, 55, 100]
labels = [ 'Joven', 'Adulto', 'Mayor']

# Categorizar la variable 'edad' con etiquetas personalizadas
df['edad_etiqueta']=pd.cut(df['edad'], bins=bins, labels=labels)
df[['edad','edad_etiqueta']]


sns.catplot(
	data=df,
 x="edad_etiqueta",
 kind="count",
 hue="diabetes"
)

### proceso cruce

In [ ]:
tabla_cruzada=pd.crosstab([df['Etiqueta_LDL'],
                           df['Etiqueta_HDL'], 
                           df['Etiqueta_Triglicéridos'], 
                           df['Etiqueta_IMC'], 
                           df['edad_etiqueta'],
                           df['genero']],
                          [df['diabetes']],dropna=False).reset_index()
tabla_cruzada=tabla_cruzada.rename(columns={0:"Sin diabetes",1:"Con diabetes"})
tabla_cruzada

In [ ]:
promedio_perimetro = df.groupby(['Etiqueta_LDL', 
                                 'Etiqueta_HDL', 
                                 'Etiqueta_Triglicéridos', 
                                 'Etiqueta_IMC', 
                                 'edad_etiqueta', 
                                 'genero',
                                 'diabetes'])['PERIMETRO_ABDOMINAL'].median().reset_index()

promedio_perimetro = pd.crosstab([promedio_perimetro['Etiqueta_LDL'],
                           promedio_perimetro['Etiqueta_HDL'], 
                           promedio_perimetro['Etiqueta_Triglicéridos'], 
                           promedio_perimetro['Etiqueta_IMC'], 
                           promedio_perimetro['edad_etiqueta'],
                           promedio_perimetro['genero']],
                          [promedio_perimetro['diabetes']],values=promedio_perimetro["PERIMETRO_ABDOMINAL"],aggfunc='sum').reset_index()

# Renombra la columna para evitar confusiones en la unión
promedio_perimetro = promedio_perimetro.rename(columns={'PERIMETRO_ABDOMINAL': 'PROMEDIO_PERIMETRO_ABDOMINAL'})
# Une el promedio del perímetro abdominal con la tabla cruzada
tabla_cruzada = pd.merge(tabla_cruzada, promedio_perimetro, 
                         on=['Etiqueta_LDL', 
                             'Etiqueta_HDL', 
                             'Etiqueta_Triglicéridos', 
                             'Etiqueta_IMC', 
                             'edad_etiqueta', 
                             'genero'], 
                         how='left')

tabla_cruzada

### Ejemplo pacientes por categoria

In [ ]:
tabla_cruzada[
	(tabla_cruzada["Etiqueta_LDL"]=="Alerta")
 &(tabla_cruzada["Etiqueta_HDL"]=="Alerta")
 &(tabla_cruzada["Etiqueta_Triglicéridos"]=="Alto")
 &(tabla_cruzada["Etiqueta_IMC"]=="Obesidad")
]

### proceso comparacion categorias

In [ ]:
data_original[data_original.HDL.between(1,20)]

In [ ]:
tabla_cruzada[tabla_cruzada["Sin diabetes"]>tabla_cruzada["Con diabetes"]]

In [ ]:
tabla_cruzada[tabla_cruzada["Sin diabetes"]<tabla_cruzada["Con diabetes"]]

In [ ]:
(1443-1338)/1443

# Diabetes

### Separacion data

In [ ]:
diabetes = df[df.diabetes==1]
diabetes

In [ ]:
no_diabetes = df[df.diabetes==0]
no_diabetes

In [ ]:
no_diabetes[no_diabetes.numero_identificacion_paciente=="1050091072"]

In [ ]:
repetidos = diabetes.numero_identificacion_paciente.value_counts()
repetidos[repetidos>1]

In [ ]:
usu_diabetes = set(diabetes.numero_identificacion_paciente)
usu_no_diabetes = set(no_diabetes.numero_identificacion_paciente)

In [ ]:
len(usu_diabetes-usu_no_diabetes)

In [ ]:
len(usu_diabetes)

In [ ]:
len(usu_no_diabetes)

### Grafica densidad

In [ ]:
sns.kdeplot(
	data=df,
 x="edad",
 hue="diabetes"
)

In [ ]:
sns.catplot(
	data=df,
 x="Etiqueta_IMC",
 kind="count",
 hue="diabetes"
)
plt.xticks(rotation=90)

In [ ]:
sns.catplot(
	data=no_diabetes,
 x="nivel_academico_paciente",
 kind="count"
)
plt.xticks(rotation=90)

In [ ]:
sns.catplot(
	data=diabetes,
 x="nivel_academico_paciente",
 kind="count"
)
plt.xticks(rotation=90)

# Modelo Naive

### implementacion modelo bayes

In [ ]:
# Definir los intervalos (bins) y las etiquetas
bins = [30, 50, 70, 90, 150]
labels = ['20-50', '51-70', '71-90', '91-']

# Categorizar la variable 'edad' con etiquetas personalizadas
df['peso_etiqueta']= pd.cut(df['peso'], bins=bins, labels=labels)

In [ ]:
# Definir los intervalos (bins) y las etiquetas
bins = [1, 1.55, 1.6, 1.65, 2]
labels = ['1-1.55', '1.56-1.6.', '1.61-1.65', '1.65-2']

# Categorizar la variable 'edad' con etiquetas personalizadas
df['talla_etiqueta']= pd.cut(df['talla'], bins=bins, labels=labels)

In [ ]:
# Definir los intervalos (bins) y las etiquetas
bins = [9, 30, 50, 70, 90,500]
labels = ['9-30', '31-50.', '35-70', '71-90', '91-']

# Categorizar la variable 'edad' con etiquetas personalizadas
df['PERIMETRO_ABDOMINAL_etiqueta']= pd.cut(df['PERIMETRO_ABDOMINAL'], bins=bins, labels=labels)

In [ ]:
# Categorizar la variable 'edad' con etiquetas personalizadas
df[["PERIMETRO_ABDOMINAL",'PERIMETRO_ABDOMINAL_etiqueta']]

In [ ]:
data_original.PERIMETRO_ABDOMINAL.describe()

In [ ]:
df.columns

In [ ]:
df_train = df[[
	'edad_etiqueta',
	'genero',
	'nivel_academico_paciente',
	'raza_paciente',
	'peso_etiqueta',
	'talla_etiqueta',
	'med_hipertension',
	'familiar_dm',
	'ant_cardiovascular',
	'dm_gestacional',
	'PERIMETRO_ABDOMINAL_etiqueta',
	'diabetes',
	'Etiqueta_HDL',
	'Etiqueta_LDL',
	'Etiqueta_Triglicéridos',
	'Etiqueta_IMC'
]]
df_train

In [ ]:
X = df_train.drop('diabetes',axis=1)
y = df_train['diabetes']

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=0)

In [ ]:
import category_encoders as ce

In [ ]:
X.columns

In [ ]:
encoder = ce.OneHotEncoder( cols=['edad_etiqueta', 'genero', 'nivel_academico_paciente', 'raza_paciente',
       'peso_etiqueta', 'talla_etiqueta', 'PERIMETRO_ABDOMINAL_etiqueta',
       'Etiqueta_HDL', 'Etiqueta_LDL', 'Etiqueta_Triglicéridos',
       'Etiqueta_IMC'])

X_train = encoder.fit_transform(X_train)
X_test = encoder.fit_transform(X_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()


gnb.fit(X_train,y_train)

In [ ]:
y_pred = gnb.predict(X_test)
y_pred

In [ ]:
y_prob = gnb.predict_proba(X_test)
y_prob

In [ ]:
y_test

In [ ]:
y_test.sum()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

# Resultados
print("Predicciones:", y_pred)
print("Probabilidades de pertenecer a cada clase:")
print(y_prob)
print(f"Precisión del modelo: {accuracy:.2f}")


# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Mostrar la matriz de confusión
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()



### Patrones toda la data

In [ ]:

sns.pairplot(df[['peso', 'talla', 'imc', 'HDL', 'LDL', 'trigliceridos',
       'PERIMETRO_ABDOMINAL','diabetes']], hue='diabetes'
             )


In [ ]:
# sns.pairplot(df[[   'nivel_academico_paciente', 'raza_paciente', 'albuminuria',
#        'Etiqueta_HDL', 'Etiqueta_LDL', 'Etiqueta_Triglicéridos',
#        'Etiqueta_IMC','diabetes']], hue='diabetes', kind='bar'
#              )


# Lista de variables categóricas
categorias = [   'nivel_academico_paciente', 'raza_paciente',
       'Etiqueta_HDL', 'Etiqueta_LDL', 'Etiqueta_Triglicéridos',
       'Etiqueta_IMC']

# Crear matriz de gráficos
for cat in categorias:
    plt.figure(figsize=(10,4))
    sns.countplot(
		data=df,
		x=cat,
		hue='diabetes'

	)
    plt.xticks(rotation=45)
    
    plt.show()

In [ ]:

df_comparaciones = {}

edad_min = df.edad.min()
edad_max = df.edad.max()


for edad_ in range(edad_min,edad_max-2):
    


	df_temp_sin_dia = df[(df.edad==edad_)&(df.diabetes==0)]
	df_temp_con_dia_0 = df[(df.edad==edad_)&(df.diabetes==1)]
	df_temp_con_dia_1 = df[(df.edad==edad_+1)&(df.diabetes==1)]
	df_temp_con_dia_2 = df[(df.edad==edad_+2)&(df.diabetes==1)]

	df_prueba = pd.concat([df_temp_sin_dia,df_temp_con_dia_0,df_temp_con_dia_1,df_temp_con_dia_2]).reset_index(drop=True)
	df_prueba['edad'] = edad_
	df_prueba['edad'] = df_temp_sin_dia["edad_etiqueta"].unique()[0]
	df_comparaciones[f'edad_{edad_}']=df_prueba

In [ ]:
df_comparaciones.keys()

In [ ]:
sns.pairplot(df_comparaciones[f'edad_{45}'][['edad','peso', 'talla', 'imc', 'HDL', 'LDL', 'trigliceridos',
       'PERIMETRO_ABDOMINAL','diabetes']], hue='diabetes')


In [ ]:
categorias = [   'nivel_academico_paciente', 'raza_paciente',
       'Etiqueta_HDL', 'Etiqueta_LDL', 'Etiqueta_Triglicéridos',
       'Etiqueta_IMC']

# Crear matriz de gráficos
for cat in categorias:
    plt.figure(figsize=(10,4))
    sns.countplot(
		data=df_comparaciones[f'edad_{45}'],
		# data=pd.concat(df_comparaciones.values()),
		x=cat,
		hue='diabetes'

	)
    plt.xticks(rotation=45)
    
    plt.show()

In [ ]:
df_comparaciones["edad_34"]

In [ ]:
df_train = pd.concat(df_comparaciones.values())[[
	'edad_etiqueta',
	'genero',
	'nivel_academico_paciente',
	'raza_paciente',
	'peso_etiqueta',
	'talla_etiqueta',
	'med_hipertension',
	'familiar_dm',
	'ant_cardiovascular',
	'dm_gestacional',
	'PERIMETRO_ABDOMINAL_etiqueta',
	'diabetes',
	'Etiqueta_HDL',
	'Etiqueta_LDL',
	'Etiqueta_Triglicéridos',
	'Etiqueta_IMC'
]]
df_train

In [ ]:
X = df_train.drop('diabetes',axis=1)
y = df_train['diabetes']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=0)

In [ ]:
encoder = ce.OneHotEncoder( cols=['edad_etiqueta', 'genero', 'nivel_academico_paciente', 'raza_paciente',
       'peso_etiqueta', 'talla_etiqueta', 'PERIMETRO_ABDOMINAL_etiqueta',
       'Etiqueta_HDL', 'Etiqueta_LDL', 'Etiqueta_Triglicéridos',
       'Etiqueta_IMC'])

X_train = encoder.fit_transform(X_train)
X_test = encoder.fit_transform(X_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()


gnb.fit(X_train,y_train)

In [ ]:
y_pred = gnb.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

# Resultados
print("Predicciones:", y_pred)
print("Probabilidades de pertenecer a cada clase:")
print(y_prob)
print(f"Precisión del modelo: {accuracy:.2f}")


# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Mostrar la matriz de confusión
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
